**Note**


#Upload Data

In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [ ]:
# Suppress SettingWithCopyWarning, nutupin warning
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/KYC/Combine_Check/Check1.csv", delimiter=';')

# # #Jika dibagi 2 excel (Jogja dan Semarang)
# df1 = pd.read_csv("/content/drive/MyDrive/KYC/Combine_Check/Check1.csv", delimiter=';')
# df2 = pd.read_csv("/content/drive/MyDrive/KYC/Combine_Check/Check2.csv", delimiter=';')
# # df3 = pd.read_csv("/content/drive/MyDrive/KYC/Combine_Check/Check3.csv", delimiter=';')

# # # Combine dataframes without keys, tambahkan df3 jika ada 3
# df = pd.concat([df1, df2], ignore_index=True)

# Reset the index to remove the extra level
df = df.reset_index(drop=True)

<ipython-input-3-b15999f58db3>:1: DtypeWarning: Columns (1,4,9,11,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/KYC/Combine_Check/Check1.csv", delimiter=';')


In [ ]:
# Print the total number of rows
total_rows = df.shape[0]
print(f'Total rows in the DataFrame: {total_rows}')

Total rows in the DataFrame: 590541


In [ ]:
# Add a new empty column named "New Column"
df['Keterangan'] = None

#Processing Data 1

In [ ]:
df.tail(10)

,ORDER ID,NO. HP,STATUS,Reason,New Remarks,ID AGENT,NAMA AGENT,Modified Date,Support DOC,Face Score,OCR,Biometric Score,WorkingDate,KTP Defect >1,Status Revoke,Reason Revoke,New Remarks Revoke,Source Data,Keterangan
590531,20240610111215800000DANAW3ID067450295614,6281276745158.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,11-06-2024 04:45:54,KK,64.0,Tidak Sesuai,0.0,11-06-2024,NaN,NaN,NaN,NaN,NaN,None
590532,20240610111215800000DANAW3ID597249690325,6285180523455.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,11-06-2024 04:46:39,Tidak Ada,77.0,Tidak Sesuai,0.0,11-06-2024,NaN,NaN,NaN,NaN,NaN,None
590533,20240610111215800000DANAW3ID186051114140,6285321126455.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,11-06-2024 04:47:12,Tidak Ada,70.0,Sesuai,0.0,11-06-2024,NaN,NaN,NaN,NaN,NaN,None
590534,20240610111215800000DANAW3ID401550782300,6285389203079.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,11-06-2024 04:47:40,Tidak Ada,77.0,Sesuai,0.0,11-06-2024,NaN,NaN,NaN,NaN,NaN,None
590535,20240610111215800000DANAW3ID194950508542,6282267043462.0,REVOKED,KTP BLUR,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,11-06-2024 04:48:08,Tidak Ada,75.0,Sesuai,0.0,11-06-2024,No,REVOKED,YOUR ID CARD PICTURE IS NOT CLEAR,Id Card Image Blur (KTP Blur),NaN,None
590536,20240610111215800000DANAW3ID320850400493,6282310973180.0,REJECT,KTP BLUR,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,11-06-2024 04:49:08,KK,64.0,Sesuai,0.0,11-06-2024,No,REVOKED,Reject Document is Not Clear and Blured (Foto ...,Id Card Image Blur (KTP Blur),NaN,None
590537,20240610111215800000DANAW3ID153250892619,628990836515.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,11-06-2024 04:49:46,Tidak Ada,78.0,Sesuai,0.0,11-06-2024,NaN,NaN,NaN,NaN,NaN,None
590538,20240610111215800000DANAW3ID300749726834,6281934578753.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,11-06-2024 04:50:17,Tidak Ada,78.0,Sesuai,0.0,11-06-2024,NaN,NaN,NaN,NaN,NaN,None
590539,20240610111215800000DANAW3ID202850998612,6282175355538.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,11-06-2024 04:51:03,Tidak Ada,68.0,Tidak Sesuai,0.0,11-06-2024,NaN,NaN,NaN,NaN,NaN,None
590540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


##Check Order Id & No Telpon

In [ ]:
def process_order_id(df):
    # Delete rows where 'ORDER ID' has less than 40 characters
    df = df[df['ORDER ID'].str.len() >= 35]

    # Check if 'ORDER ID' length is less than 40
    df.loc[df['ORDER ID'].str.len() < 40, 'Keterangan'] = 'order id salah'

    # Update 'Keterangan' for 'ORDER ID' with more than 40 characters
    df.loc[df['ORDER ID'].str.len() > 40, 'Keterangan'] = df.apply(
        lambda row: row['Keterangan'] + ' & order id salah' if pd.notna(row['Keterangan']) else 'order id salah',
        axis=1
    )

    # Check 'NO. HP' and update 'Keterangan' if the condition is not met
    df.loc[~df['NO. HP'].astype(str).str.startswith(('6', '8', '0')), 'Keterangan'] = df.apply(
        lambda row: row['Keterangan'] + ' & Telpon Check' if pd.notna(row['Keterangan']) else 'no telpon salah',
        axis=1
    )

    return df

In [ ]:
# Call the function with the DataFrame
df = process_order_id(df)

In [ ]:
# View rows where 'Keterangan' is not null
result_df = df[df['Keterangan'].notnull()]

# Display the result
result_df

,ORDER ID,NO. HP,STATUS,Reason,New Remarks,ID AGENT,NAMA AGENT,Modified Date,Support DOC,Face Score,OCR,Biometric Score,WorkingDate,KTP Defect >1,Status Revoke,Reason Revoke,New Remarks Revoke,Source Data,Keterangan
260115,20240609111215800000DANAW3ID035649259761,NaN,CHECK,NaN,NaN,fatikayulianingrum@radikari.com,Fatika Yulianingrum,10-06-2024 06:09:14,Tidak Ada,73.0,Sesuai,0.0,10-06-2024,NaN,NaN,NaN,NaN,NaN,no telpon salah


In [ ]:
ori_df = df

##Formatting Date & Delete Source Data

In [ ]:
df.tail()

,ORDER ID,NO. HP,STATUS,Reason,New Remarks,ID AGENT,NAMA AGENT,Modified Date,Support DOC,Face Score,OCR,Biometric Score,WorkingDate,KTP Defect >1,Status Revoke,Reason Revoke,New Remarks Revoke,Source Data,Keterangan
590535,20240610111215800000DANAW3ID194950508542,6282267043462.0,REVOKED,KTP BLUR,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,11-06-2024 04:48:08,Tidak Ada,75.0,Sesuai,0.0,11-06-2024,No,REVOKED,YOUR ID CARD PICTURE IS NOT CLEAR,Id Card Image Blur (KTP Blur),NaN,None
590536,20240610111215800000DANAW3ID320850400493,6282310973180.0,REJECT,KTP BLUR,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,11-06-2024 04:49:08,KK,64.0,Sesuai,0.0,11-06-2024,No,REVOKED,Reject Document is Not Clear and Blured (Foto ...,Id Card Image Blur (KTP Blur),NaN,None
590537,20240610111215800000DANAW3ID153250892619,628990836515.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,11-06-2024 04:49:46,Tidak Ada,78.0,Sesuai,0.0,11-06-2024,NaN,NaN,NaN,NaN,NaN,None
590538,20240610111215800000DANAW3ID300749726834,6281934578753.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,11-06-2024 04:50:17,Tidak Ada,78.0,Sesuai,0.0,11-06-2024,NaN,NaN,NaN,NaN,NaN,None
590539,20240610111215800000DANAW3ID202850998612,6282175355538.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,11-06-2024 04:51:03,Tidak Ada,68.0,Tidak Sesuai,0.0,11-06-2024,NaN,NaN,NaN,NaN,NaN,None


In [ ]:
# df['NO. HP'] = df['NO. HP'].astype('int64')
df['ORDER ID'] = df['ORDER ID'].astype('str')

#date
# df['Modified Date'] = pd.to_datetime(df['Modified Date'], errors='coerce')
df['Modified Date'] = pd.to_datetime(df['Modified Date'], format='%d-%m-%Y %H:%M:%S', errors='coerce')

df['WorkingDate'] = pd.to_datetime(df['WorkingDate'], format='%d-%m-%Y', errors='coerce')
#df['WorkingDate'] = df['WorkingDate'].dt.strftime('%Y-%m-%d')

# the column containing extra spaces
df['ORDER ID'] = df['ORDER ID'].str.strip()

# df['NO. HP'] = df['NO. HP'].astype('int64')
df['Face Score'] = df['Face Score'].astype('str')

df.drop('Source Data', axis=1, inplace=True)
df2=df


In [ ]:
df.tail()

,ORDER ID,NO. HP,STATUS,Reason,New Remarks,ID AGENT,NAMA AGENT,Modified Date,Support DOC,Face Score,OCR,Biometric Score,WorkingDate,KTP Defect >1,Status Revoke,Reason Revoke,New Remarks Revoke,Keterangan
590535,20240610111215800000DANAW3ID194950508542,6282267043462.0,REVOKED,KTP BLUR,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,2024-06-11 04:48:08,Tidak Ada,75.0,Sesuai,0.0,2024-06-11,No,REVOKED,YOUR ID CARD PICTURE IS NOT CLEAR,Id Card Image Blur (KTP Blur),None
590536,20240610111215800000DANAW3ID320850400493,6282310973180.0,REJECT,KTP BLUR,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,2024-06-11 04:49:08,KK,64.0,Sesuai,0.0,2024-06-11,No,REVOKED,Reject Document is Not Clear and Blured (Foto ...,Id Card Image Blur (KTP Blur),None
590537,20240610111215800000DANAW3ID153250892619,628990836515.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,2024-06-11 04:49:46,Tidak Ada,78.0,Sesuai,0.0,2024-06-11,NaN,NaN,NaN,NaN,None
590538,20240610111215800000DANAW3ID300749726834,6281934578753.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,2024-06-11 04:50:17,Tidak Ada,78.0,Sesuai,0.0,2024-06-11,NaN,NaN,NaN,NaN,None
590539,20240610111215800000DANAW3ID202850998612,6282175355538.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,2024-06-11 04:51:03,Tidak Ada,68.0,Tidak Sesuai,0.0,2024-06-11,NaN,NaN,NaN,NaN,None


###Melakukan filter Start Date & End Date pada Working Date

In [ ]:
oldest_date = df['WorkingDate'].min()
latest_date = df['WorkingDate'].max()
print("The oldest date is:", oldest_date)
print("The oldest date is:", latest_date)

The oldest date is: 2024-06-01 00:00:00
The oldest date is: 2024-06-11 00:00:00


In [ ]:
from google.colab import files

try:
  # Take user input for start and end dates
  print("The oldest date is:", oldest_date)
  print("The oldest date is:", latest_date)
  start_date = input("Enter start date (YYYY-MM-DD): ")
  end_date = input("Enter end date (YYYY-MM-DD): ")

  # Convert input dates to datetime objects
  start_date = pd.to_datetime(start_date)
  end_date = pd.to_datetime(end_date)

  # Filter DataFrame based on the input dates
  df = df[(df['WorkingDate'] >= start_date) & (df['WorkingDate'] <= end_date)]

except ValueError as e:
    # Handle the ValueError (e.g., invalid date format)
    print(f"Error: {e}. Please enter dates in the 'YYYY-MM-DD' format.")
except Exception as e:
    # Handle other exceptions
    print(f"An unexpected error occurred: {e}")

The oldest date is: 2024-06-01 00:00:00
The oldest date is: 2024-06-11 00:00:00
Enter start date (YYYY-MM-DD): 2024-6-1
Enter end date (YYYY-MM-DD): 2024-6-10


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 549892 entries, 0 to 590235
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   ORDER ID            549892 non-null  object        
 1   NO. HP              549891 non-null  object        
 2   STATUS              549866 non-null  object        
 3   Reason              81502 non-null   object        
 4   New Remarks         7548 non-null    object        
 5   ID AGENT            549892 non-null  object        
 6   NAMA AGENT          549892 non-null  object        
 7   Modified Date       549742 non-null  datetime64[ns]
 8   Support DOC         549738 non-null  object        
 9   Face Score          549892 non-null  object        
 10  OCR                 549729 non-null  object        
 11  Biometric Score     546642 non-null  object        
 12  WorkingDate         549892 non-null  datetime64[ns]
 13  KTP Defect >1       81511 non-null

In [ ]:
df.head()

,ORDER ID,NO. HP,STATUS,Reason,New Remarks,ID AGENT,NAMA AGENT,Modified Date,Support DOC,Face Score,OCR,Biometric Score,WorkingDate,KTP Defect >1,Status Revoke,Reason Revoke,New Remarks Revoke,Keterangan
0,20240601111215800000DANAW3ID017442453747,83833991865,CHECK,NaN,NaN,agent.jogja769@radikari.com,Galih Reka Mahendra,2024-06-01 11:03:04,KK,67.0,Tidak Sesuai,0,2024-06-01,NaN,NaN,NaN,NaN,None
1,20240601111215800000DANAW3ID526442394329,85343578803,CHECK,NaN,NaN,agent.jogja511@radikari.com,Hesti Vera,2024-06-02 08:08:09,KK,73.0,Tidak Sesuai,0,2024-06-02,NaN,NaN,NaN,NaN,None
2,20240603111215800000DANAW3ID345644368110,85198562003,CHECK,NaN,NaN,agent.jogja511@radikari.com,Hesti Vera,2024-06-03 11:43:30,KK,75.0,Sesuai,0,2024-06-03,NaN,NaN,NaN,NaN,None
3,20240603111215800000DANAW3ID756844255430,81226168970,CHECK,NaN,NaN,agent.jogja609@radikari.com,Amalia Nurhayati,2024-06-03 12:45:51,KK,77.0,Tidak Sesuai,0,2024-06-03,NaN,NaN,NaN,NaN,None
4,20240604111215800000DANAW3ID270346001519,87812994719,CHECK,NaN,NaN,agent.jogja769@radikari.com,Galih Reka Mahendra,2024-06-04 11:40:19,SIM,63.0,Sesuai,0,2024-06-04,NaN,NaN,NaN,NaN,None


## Check Status

In [ ]:
def update_status(row):

    if pd.isna(row['STATUS']) and row['WorkingDate'] == end_date:
      return row['Keterangan']

    elif pd.isna(row['STATUS']) or row['STATUS'] not in ['CHECK', 'REJECT', 'REVOKED','REJECT BY SYSTEM']:
      if pd.isna(row['Keterangan']):
        return 'check status'
      else:
        return f"{row['Keterangan']} & check status"

    elif row['STATUS'] == 'CHECK' and pd.notna(row['Reason']):
      if pd.isna(row['Keterangan']):
        return 'check status'
      else:
        return f"{row['Keterangan']} & check status"

    elif row['STATUS'] in ['REJECT', 'REVOKED'] and pd.isna(row['Reason']):
      if pd.isna(row['Keterangan']):
        return 'check status'
      else:
        return f"{row['Keterangan']} & check status"

    # elif row['STATUS'] in ['REJECT BY SYSTEM'] and pd.notna(row['Reason']) and and pd.notna(row['Reason']):
    #   if pd.isna(row['Keterangan']):
    #     return 'check status'
    #   else:
    #     return f"{row['Keterangan']} & check status"

    else:
        return row['Keterangan']

In [ ]:
# Apply the custom function to update "Keterangan" column
df['Keterangan'] = df.apply(update_status, axis=1)

In [ ]:
# Remove values in 'Keterangan' column where 'STATUS' is "REJECT BY SYSTEM"
df.loc[df['STATUS'] == 'REJECT BY SYSTEM', 'Keterangan'] = np.nan

##NEW Check ID Agent

In [ ]:
# rumus excel:  =TEXTJOIN(", "; TRUE; """" & A1:A250 & """ : """ & B1:B250 & """")
# Dictionary of agent IDs and names
agent_dict = {
"agent.jogja403@radikari.com" : "Yunita Risang Saksana Awiati", "agent.jogja407@radikari.com" : "Wibulan Ekaning Ratri", "agent.jogja418@radikari.com" : "Krisna Wijaya", "agent.jogja435@radikari.com" : "Ryan Oktavianto", "agent.jogja438@radikari.com" : "Chatarina Ria Septiana", "agent.jogja464@radikari.com" : "Wiwik Suharti", "agent.jogja487@radikari.com" : "Teguh Bagus Joko Wahono", "agent.jogja511@radikari.com" : "Hesti Vera", "agent.jogja517@radikari.com" : "Arlinda Elsa Ulvia", "agent.jogja566@radikari.com" : "Eri Hardani", "agent.jogja609@radikari.com" : "Amalia Nurhayati", "agent.jogja615@radikari.com" : "Puji Kasih Santika", "agent.jogja674@radikari.com" : "Wiwid prastianto", "agent.jogja679@radikari.com" : "Fathur Rohman Yunus", "agent.jogja738@radikari.com" : "Yudha Tantra Hanurizal", "agent.jogja740@radikari.com" : "Bobby Yulianto", "agent.jogja745@radikari.com" : "Rio Jati Kumbara", "agent.jogja749@radikari.com" : "Vincentius Juan Fergi Hermantya", "agent.jogja751@radikari.com" : "Ahda Rasyidi Ahmad", "agent.jogja769@radikari.com" : "Galih Reka Mahendra", "agent.jogja770@radikari.com" : "Galih Fajar Kurniawan", "agent.jogja784@radikari.com" : "Rika Ferawati", "agent.jogja814@radikari.com" : "Regitha Aprilia Ochie", "agent.jogja876@radikari.com" : "Yuriatun", "amandarapih@radikari.com" : "Amanda Rapih Subarja Putri", "nabilakhansa@radikari.com" : "Nabila Khansa Afanin", "paiznurdiansyah@radikari.com" : "Paiz Nurdiansyah", "yuliastutik@radikari.com" : "Yuli Astutik", "agent.jogja897@radikari.com" : "Afwan Hilmi Muttaqin", "agent.jogja898@radikari.com" : "Dita Sukmawati Putri", "agent.jogja899@radikari.com" : "Elysama Tamba", "agent.jogja900@radikari.com" : "Ihsan Pandu Tauladan", "agent.jogja902@radikari.com" : "Mahmudi Waruwu", "agent.jogja903@radikari.com" : "Ratih Ningtiyas", "agent.jogja904@radikari.com" : "Ratna Sri Handayani", "agent.jogja905@radikari.com" : "Ridhol Fatla", "agent.jogja907@radikari.com" : "Septi Sindisat Rahayu", "agent.jogja908@radikari.com" : "Siti Ulfah Hardiyanti", "agent.jogja909@radikari.com" : "Slamet Setiyono", "agent.jogja910@radikari.com" : "Topan Jiwodewo", "agent.jogja912@radikari.com" : "Dea Fatika Putri", "agent.jogja913@radikari.com" : "Ibnu Hajar Aswad", "agent.jogja914@radikari.com" : "Indira Pradipta", "agent.jogja916@radikari.com" : "Luh Putu Nindya Meichintya Dewi", "agent.jogja917@radikari.com" : "Raden Kurnianto Ridwan Hanafi", "agent.jogja918@radikari.com" : "Randy Aldany Barasa", "agent.jogja920@radikari.com" : "Satria Bobby Fernando", "agent.jogja921@radikari.com" : "Stevanus Bagas Pramudita", "agent.jogja922@radikari.com" : "Vivia Tri Ayuningtyas", "agent.jogja924@radikari.com" : "Zarah Usra", "agent.jogja925@radikari.com" : "Abiyan Roofif Mahendra", "agent.jogja926@radikari.com" : "Aryani Wuri Aningrum", "agent.jogja927@radikari.com" : "Dama Agus Setiawan", "agent.jogja928@radikari.com" : "Donie Pramudita", "agent.jogja929@radikari.com" : "Eka Edi Pratama", "agent.jogja930@radikari.com" : "Firman Syahputra", "agent.jogja931@radikari.com" : "Hanifah Argiyanti Kusumaningrum", "agent.jogja932@radikari.com" : "Irvan Susilo Widjiatmoko", "agent.jogja933@radikari.com" : "Kevin Aditya Mahardhika", "agent.jogja934@radikari.com" : "Ricy Ardiansyah", "agent.jogja935@radikari.com" : "Achmad Miftahul Ihsan Fauzi", "agent.jogja936@radikari.com" : "Adjie Bhagyawan Prasetyo", "agent.jogja937@radikari.com" : "Andreas Suryo Prasetyo", "agent.jogja938@radikari.com" : "Azhar Hani Prihantoro", "agent.jogja940@radikari.com" : "Krisna Dewa Nanda", "agent.jogja941@radikari.com" : "Lutfi Hariyanto Nugroho", "agent.jogja942@radikari.com" : "Mella Sutami", "agent.jogja944@radikari.com" : "Naufal Ariq Suryawan", "agent.jogja945@radikari.com" : "Rido Mesia Sumunar", "agent.jogja946@radikari.com" : "Wahyu Adi Purnomo", "agent.jogja947@radikari.com" : "Wanti Rista Anggraini", "agent.jogja948@radikari.com" : "Yokasanu Ahdiat", "agent.jogja949@radikari.com" : "Cahyo Adhi Prabowo", "agent.jogja950@radikari.com" : "Ehsan Alfian Pratama", "agent.jogja951@radikari.com" : "Lailatul Rahmah", "agent.jogja952@radikari.com" : "Lintang Wahyu Herlambang", "agent.jogja953@radikari.com" : "Manggala Maulana Mahardhika", "agent.jogja954@radikari.com" : "Ngrayong Agung Nugroho", "agent.jogja955@radikari.com" : "Nova Putra Alves Correia", "agent.jogja956@radikari.com" : "Nurul Istikomah", "agent.jogja957@radikari.com" : "Pratiwi Matoka", "agent.jogja958@radikari.com" : "Yessy Artileriyana Shinta", "agent.jogja959@radikari.com" : "Yulian Nur Jayanto", "agent.jogja967@radikari.com" : "Rean Prandika Henga", "alfredo.yunara@radikari.com" : "Alfredo Yunora", "arbinugraha@radikari.com" : "Arbi Tama Nugraha", "ariffabikhan@radikari.com" : "Arif Fabikhan", "bagusramadhan@radikari.com" : "Bagus Ramadhan Firdausi", "bayuasmoro@radikari.com" : "Bayu Asmoro Putro", "bimasatriyapahlawan@radikari.com" : "Bima Satriya Pahlawan", "cintapratiwi@radikari.com" : "Cinta Pratiwi Taasiringan", "dandiaditya@radikari.com" : "Dandi Aditya Rahmanto", "davidfauzan@radikari.com" : "David Fauzan", "dewipujilestari@radikari.com" : "Dewi Puji Lestari", "deyskeerika@radikari.com" : "Deyske Erika Rembaen", "dhenisafdhaly@radikari.com" : "Dheni Nandar Safdhaly", "ekonurjayadi@radikari.com" : "Eko Nur Jayadi", "faliaputri@radikari.com" : "Falia Rahmazati Putri", "fatikayulianingrum@radikari.com" : "Fatika Yulianingrum", "hanifalvian@radikari.com" : "Hanif Alvian Rosyiadi", "haryoajibandera@radikari.com" : "Haryo Aji Bandera", "khusnulridho@radikari.com" : "Khusnul Ridho Moh Arif", "linamakarim@radikari.com" : "Lina Makarim", "lovianaikasari@radikari.com" : "Loviana Ika Sari", "martinus.beko@radikari.com" : "Martinus Beko", "muhammad.anwarudin@radikari.com" : "Muhammad Anwarudin", "n.agungpermana@radikari.com" : "Nuro'is Agung Permana", "nadiairana@radikari.com" : "Nadia Irana", "nonyaffi@radikari.com" : "Nony mariani affi", "paraldisbastian@radikari.com" : "Paraldi Sbastian Bagaskara", "radenardimas@radikari.com" : "Raden Ardimas Suryo Prabowo", "rantisulistiyoningsih@radikari.com" : "Ranti Sulistiyoningsih", "rasiti@radikari.com" : "Raden Ajeng Siti Nurul Erika Farida", "reckybranada@radikari.com" : "Recky Branada", "saniayolanda@radikari.com" : "Sania Yolanda", "sasalanthika@radikari.com" : "Sasa Lanthika", "sitinoordiwanti@radikari.com" : "Siti Noor Diwanti", "syamsudinbadawi@radikari.com" : "Syamsudin Badawi", "tomyprabowo@radikari.com" : "Tomy Ady Prabowo", "trisiswiharjanti@radikari.com" : "Tri Siswi Harjanti", "ummihamidah@radikari.com" : "Ummi Hamidah Kartikasari", "wahyusetiawan@radikari.com" : "Wahyu Setiawan", "ahmadbukhori@radikari.com" : "Ahmad Bukhori", "albertngobut@radikari.com" : "Albert Pieter Ngobut", "betijulianti@radikari.com" : "Beti Indri Julianti", "danangpranowo@radikari.com" : "Danang Wahyu Pranowo", "deaningrum@radikari.com" : "Dea Ayu Septia Ningrum", "diasarnanda@radikari.com" : "Dias Arnanda", "isnaeniridha@radikari.com" : "Isnaeni Ridha A", "salmafebiola@radikari.com" : "Salma Silvia Febiola", "wulansyaidah@radikari.com" : "Wulan Dewi Fatikhatus Syaidah", "adeismoyowati@radikari.com" : "Ade Ismoyowati Setiyono", "agent.srg001@radikari.com" : "Hamzah Fakhrurozi", "agent.srg003@radikari.com" : "Yoditya Iqbal Respati", "agent.srg005@radikari.com" : "Dewi Murwati", "agent.srg032@radikari.com" : "Nanda Puspa Anisya", "agent.srg037@radikari.com" : "Dody Riza Permadi", "agent.srg056@radikari.com" : "Eka Cynthia Yuliawan", "agent.srg072@radikari.com" : "Syarifudin Fathur", "agent.srg076@radikari.com" : "Monica Yunda Permatasari", "agent.srg105@radikari.com" : "Ririn Isti Qomah", "agent.srg141@radikari.com" : "Fany Slamet Sulastri", "agent.srg343@radikari.com" : "Mahendra Harima Putra", "agent.srg431@radikari.com" : "Irvan Mahendra", "agent.srg438@radikari.com" : "Novan Lestari Putra", "ajengwidyaningsih@radikari.com" : "Ajeng Widyaningsih", "yunizafakmal@radikari.com" : "Yunizaf Akmal Bachtiar", "agent.srg509@radikari.com" : "Yuliana", "agent.srg510@radikari.com" : "Wina Putri Oktavia", "agent.srg511@radikari.com" : "Damara Galas Saputro", "agent.srg512@radikari.com" : "Bakhtiar Seno Aji", "agent.srg513@radikari.com" : "Achmad Khoirul", "agent.srg514@radikari.com" : "Millenia Sofianti", "agent.srg515@radikari.com" : "Umi Khofiroh", "agent.srg516@radikari.com" : "Rio Hadi Susilo", "agent.srg520@radikari.com" : "Bagas Surya Dewantoro", "agent.srg521@radikari.com" : "Suci Ananda Sari", "agent.srg522@radikari.com" : "Randira Aulia", "agent.srg523@radikari.com" : "Argia Asmara Dewanggoro", "agent.srg524@radikari.com" : "Adella Titah Anggita", "agent.srg525@radikari.com" : "Rahadian Ichsan Pratama", "agent.srg526@radikari.com" : "Hendri Septawidya Pratama", "agent.srg527@radikari.com" : "Dias Nugroho", "agent.srg528@radikari.com" : "Dhila Muhammad Ali", "agent.srg529@radikari.com" : "Muhammad Iqbal Muhyiddin", "agent.srg530@radikari.com" : "Aulia Rachma Yanuarita", "agent.srg531@radikari.com" : "Catur Prasetya", "agent.srg532@radikari.com" : "Siti Hartinah", "agent.srg533@radikari.com" : "Johan Hamuda", "agent.srg534@radikari.com" : "Anthony Wijaya", "agent.srg535@radikari.com" : "Tri Adi Wijaya", "agent.srg536@radikari.com" : "Mohammad Fikri Mudzaki", "agiar.asmara@radikari.com" : "Agiar Asmara Bagaskoro", "ahmad.fakhruri@radikari.com" : "Ahmad Ali Fakhruri", "amynaufal@radikari.com" : "Amy Naufal Falah", "anasaidatin@radikari.com " : "Ana Saidatin", "angga.ramadhan@radikari.com" : "Angga Ramadhan Sieko Putra", "arimurtiningtyas@radikari.com" : "Ari Murtiningtyas", "atiknasekha@radikari.com" : "Atik Dina Nasekha", "aysiliaratnasari@radikari.com" : "Aysilia Audy Ratnasari", "cakratriatmaja@radikari.com" : "Cakra Triatmaja", "daityo@radikari.com" : "Dhimas Adityo Nurhayat", "devipujiseptiyani@radikari.com" : "Devi Puji Septiyani", "edosaputra@radikari.com" : "Edo Hidayatullah Saputra", "fahriidhofi@radikari.com" : "Fahri Idhofi", "farahautami@radikari.com" : "Farah Alma Utami", "firdamaudy@radikari.com" : "Firda Maudy Salsabilla", "galiehsaputra@radikari.com" : "Galih Wahyu Hadi Saputra", "gilang.pratama@radikari.com" : "Gilang Dika Pratama", "gilangadi@radikari.com" : "Gilang Adi Pratama", "hafiattaqi@radikari.com" : "Hafi Attaqi Rumi", "hafidalaziz@radikari.com" : "Hafid Alaziz", "hanifalbab@radikari.com" : "Hanif Albab", "hanung@radikari.com" : "Husein Anung Anindhito", "hriefandi@radikari.com" : "Hilman Riefandi", "isnaeni.trisna@radikari.com" : "Isnaeni Trisna Rahmawati", "mahabbatulmaula@radikari.com" : "Mahabbatul Maula", "mayhusnaino@radikari.com" : "Mayhusna Ino Tantyasari", "mluqmanhakim@radikari.com" : "Muhammad Luqman Hakim Abdullah", "Muhammad.faisal@radikari.com" : "Muhammad Faisal Amri", "muhammad.mudi@radikari.com" : "M. Mudi", "muhammadvarian@radikari.com" : "Muhammad Farell Varian", "mvennesa@radikari.com" : "Mellania Vennesa Fitri", "nadya.yun@radikari.com" : "Nadya Yun Tafa Luwih", "nafidzailmi@radikari.com" : "Nafidza Ilmi Khairina", "natanielaxl@radikari.com" : "Nataniel Axl Areza", "novitaqiroati@radikari.com" : "Novita Qiroati Fadilah", "nurzilzabahri@radikari.com" : "Nur Zilza Bunayya Bahri", "nyasykur@radikari.com" : "Naufal Yasykur Odi Pindo", "oktariskasalas@radikari.com" : "Oktariska Salas Niko", "rafandi@radikari.com" : "Rosyad Afandi", "⁠reza.adjie@radikari.com" : "Reza Purnama Adjie", "riyanita.nita@radikari.com" : "Riyanita", "sarpini.narmi@radikari.com" : "Sarpini", "satryaajisepta@radikari.com" : "Satrya Aji Septa Buana", "sitichoiriyah@radikari.com" : "Siti Choiriyah", "wahyu.setyawan@radikari.com" : "Wahyu Setyawan", "yudwita.rahmana@radikari.com" : "Yudwita Rohmana", "yuliadwi@radikari.com" : "Yulia Dwi Hardyanti", "yunan.azzumardi@radikari.com" : "Yunan Azzumardi Irawan", "ahmadkarim@radikari.com" : "Ahmad Busyrol Karim", "fitriasubekti@radikari.com" : "Fitria Nuraini Subekti", "galihsari@radikari.com" : "Galih Ayu Wulan Sari", "kalistasedha@radikari.com" : "Kalista Sia Sedha", "miftahulqoyim@radikari.com" : "Muhammad Miftahul Qoyim", "rizzafahlevi@radikari.com" : "Muhammad Rizza Fahlevi", "saidihusaini@radikari.com" : "Muhammad Saidi Husaini MZ", "zakyulinnuha@radikari.com" : "Muhammad Zaky Ulinnuha", "nadiaamalina@radikari.com" : "Nadia Rizki Amalina", "ninengrahayu@radikari.com" : "Nineng Sri Rahayu", "rendyadriansyah@radikari.com" : "Rendy Adriansyah", "varellkhudhori@radikari.com" : "Varell Abdul Rozaq Khudhori", "reggawijaya@radikari.com" : "W. Regga Wijaya", "wahyusetiyawan@radikari.com" : "Wahyu Setiyawan", "yusufhutomo@radikari.com" : "Yusuf Murwido Hutomo"

}


In [ ]:
# Print the table header
print("| ID Agent                        | Name                       |")
print("|--------------------------------|----------------------------|")

# Print each row of the table
for id_agent, name in agent_dict.items():
    print(f"| {id_agent:<30} | {name:<26} |")

| ID Agent                        | Name                       |
|--------------------------------|----------------------------|
| agent.jogja403@radikari.com    | Yunita Risang Saksana Awiati |
| agent.jogja407@radikari.com    | Wibulan Ekaning Ratri      |
| agent.jogja418@radikari.com    | Krisna Wijaya              |
| agent.jogja435@radikari.com    | Ryan Oktavianto            |
| agent.jogja438@radikari.com    | Chatarina Ria Septiana     |
| agent.jogja464@radikari.com    | Wiwik Suharti              |
| agent.jogja487@radikari.com    | Teguh Bagus Joko Wahono    |
| agent.jogja511@radikari.com    | Hesti Vera                 |
| agent.jogja517@radikari.com    | Arlinda Elsa Ulvia         |
| agent.jogja566@radikari.com    | Eri Hardani                |
| agent.jogja609@radikari.com    | Amalia Nurhayati           |
| agent.jogja615@radikari.com    | Puji Kasih Santika         |
| agent.jogja674@radikari.com    | Wiwid prastianto           |
| agent.jogja679@radikari.com    | Fa

In [ ]:
# for idx, row in df.iterrows():
#     if row['ID AGENT'] == '':
#       if pd.isna(row['Keterangan']) or row['Keterangan'] == '':
#         df.at[idx, 'Keterangan'] = 'ID Agent Salah'
#       else:
#         df.at[idx, 'Keterangan'] = f"{row['Keterangan']} & ID Agent Salah"

#     elif row['ID AGENT'] not in agent_dict:
#         if row['NAMA AGENT'] in agent_dict.values():
#             # Replace id agent with dict id agent
#             for k, v in agent_dict.items():
#                 if v == row['NAMA AGENT']:
#                     df.at[idx, 'ID AGENT'] = k
#                     if pd.isna(row['Keterangan']) or row['Keterangan'] == '':
#                       df.at[idx, 'Keterangan'] = 'ID Agent Salah'
#                     else:
#                       df.at[idx, 'Keterangan'] = f"{row['Keterangan']} & ID Agent Salah"
#                       break
#         else:
#             if df.at[idx, 'Keterangan'] == "":
#                 df.at[idx, 'Keterangan'] = 'ID Agent & Nama Agent Salah'
#             else:
#              df.at[idx, 'Keterangan'] = f"{row['Keterangan']} & ID Agent Salah & Nama Agent Salah"

#     elif row['NAMA AGENT'] != agent_dict[row['ID AGENT']]:
#         # Replace agent name with dict agent name
#         df.at[idx, 'NAMA AGENT'] = agent_dict[row['ID AGENT']]
#         if df.at[idx, 'Keterangan'] == "":
#             df.at[idx, 'Keterangan'] = 'Nama Agent Salah'
#         else:
#          df.at[idx, 'Keterangan'] = f"{row['Keterangan']} & Nama Agent Salah"


In [ ]:
for idx, row in df.iterrows():
    current_ket = df.at[idx, 'Keterangan'] if not pd.isna(df.at[idx, 'Keterangan']) else ""

    if row['ID AGENT'] == '':
        df.at[idx, 'Keterangan'] = 'ID Agent Salah' if current_ket == "" else f"{current_ket} & ID Agent Salah"

    elif row['ID AGENT'] not in agent_dict:
        if row['NAMA AGENT'] in agent_dict.values():
            match_id = next((k for k, v in agent_dict.items() if v == row['NAMA AGENT']), None)
            if match_id:
                df.at[idx, 'ID AGENT'] = match_id
                df.at[idx, 'Keterangan'] = 'ID Agent Salah' if current_ket == "" else f"{current_ket} & ID Agent Salah"
        else:
            df.at[idx, 'Keterangan'] = 'ID Agent & Nama Agent Salah' if current_ket == "" else f"{current_ket} & ID Agent Salah & Nama Agent Salah"

    elif row['NAMA AGENT'] != agent_dict.get(row['ID AGENT'], ''):
        df.at[idx, 'NAMA AGENT'] = agent_dict[row['ID AGENT']]
        df.at[idx, 'Keterangan'] = 'Nama Agent Salah' if current_ket == "" else f"{current_ket} & Nama Agent Salah"

In [ ]:
filtered_data = df[df['Keterangan'].notnull()]
filtered_data.head(10)

,ORDER ID,NO. HP,STATUS,Reason,New Remarks,ID AGENT,NAMA AGENT,Modified Date,Support DOC,Face Score,OCR,Biometric Score,WorkingDate,KTP Defect >1,Status Revoke,Reason Revoke,New Remarks Revoke,Keterangan
200325,20240609111215800000DANAW3ID487750241174,628895804456,NaN,NaN,NaN,agent.jogja928@radikari.com,Donie Pramudita,2024-06-09 22:36:09,Tidak Ada,73,Sesuai,0,2024-06-09,NaN,NaN,NaN,NaN,check status
238981,20240607111215800000DANAW3ID084947852646,6285229097824,REVOKED,NaN,NaN,agent.jogja955@radikari.com,Nova Putra Alves Correia,2024-06-08 01:41:13,Tidak Ada,71.0,Tidak Sesuai,0.0,2024-06-08,NaN,NaN,NaN,NaN,check status
239204,20240607111215800000DANAW3ID736947529667,6285832023157,CHECK,SELFIE BERBEDA,NaN,agent.jogja955@radikari.com,Nova Putra Alves Correia,2024-06-08 04:51:23,KK,57.0,Tidak Sesuai,0.0,2024-06-08,NaN,NaN,NaN,NaN,check status
260115,20240609111215800000DANAW3ID035649259761,NaN,CHECK,NaN,NaN,fatikayulianingrum@radikari.com,Fatika Yulianingrum,2024-06-10 06:09:14,Tidak Ada,73.0,Sesuai,0.0,2024-06-10,NaN,NaN,NaN,NaN,no telpon salah
263805,20240608111215800000DANAW3ID573047759841,6285321307678,CHECK,NaN,NaN,ahmadbukhori@radikari.com,Ahmad Bukhori,2024-06-08 23:45:00,Tidak Ada,78.0,Tidak Sesuai,0.0,2024-06-08,NaN,NaN,NaN,NaN,ID Agent Salah
283565,20240610111215800000DANAW3ID724649577419,6287728421285,CHECK,SELFIE BERBEDA,NaN,ahmadkarim@radikari.com,Ahmad Busyrol Karim,2024-06-10 15:19:43,Tidak Ada,57.0,Tidak Sesuai,0.0,2024-06-10,No,REVOKED,Reject Selfie doesn't match with ID Photo (Sel...,Selfie Doesn’t Match With Id Card (Selfie tida...,check status
287804,20240610111215800000DANAW3ID935149309954,6285850462938,REJECT,NaN,NaN,miftahulqoyim@radikari.com,Muhammad Miftahul Qoyim,2024-06-10 16:35:15,KK,56.0,Sesuai,0.0,2024-06-10,No,REVOKED,Reject Selfie doesn't match with ID Photo (Sel...,Selfie Doesn’t Match With Id Card (Selfie tida...,check status
297511,20240610111215800000DANAW3ID604250342226,62882020814464,NaN,NaN,NaN,varellkhudhori@radikari.com,Varell Abdul Rozaq Khudhori,NaT,NaN,nan,NaN,0.0,2024-06-09,NaN,NaN,NaN,NaN,check status
297512,20240610111215800000DANAW3ID888250449968,6282284073737,NaN,NaN,NaN,varellkhudhori@radikari.com,Varell Abdul Rozaq Khudhori,NaT,NaN,nan,NaN,0.0,2024-06-09,NaN,NaN,NaN,NaN,check status
297513,20240610111215800000DANAW3ID163350583874,6283824340910,NaN,NaN,NaN,varellkhudhori@radikari.com,Varell Abdul Rozaq Khudhori,NaT,NaN,nan,NaN,0.0,2024-06-09,NaN,NaN,NaN,NaN,check status


## Check Modified Date & Working Date

In [ ]:
# Create a custom function to check conditions and update "Keterangan" column
def update_keterangan(row):
    if pd.isna(row['Modified Date']):
      if pd.isna(row['Keterangan']):
        return 'modified date salah'
      else:
        return f"{row['Keterangan']} & modified date salah"

    elif row['Modified Date'].date() != row['WorkingDate'].date():
        if pd.isna(row['Keterangan']):
            return 'working date salah'
        else:
            return f"{row['Keterangan']} & working date salah"
    else:
        return row['Keterangan']

# Apply the custom function to update "Keterangan" column
df['Keterangan'] = df.apply(update_keterangan, axis=1)

#Change The Data

##Mengganti Nomor Telpon

In [ ]:
pd.set_option('display.max_rows', None)
telpon = df[df['Keterangan'].str.contains('telpon', case=False, na=False)]
telpon[['ORDER ID','NO. HP', 'Modified Date','WorkingDate','Keterangan']]


,ORDER ID,NO. HP,Modified Date,WorkingDate,Keterangan
260115,20240609111215800000DANAW3ID035649259761,NaN,2024-06-10 06:09:14,2024-06-10,no telpon salah


In [ ]:
# Define a base URL
base_url = "https://custmng.saas.dana.id/user/kyc/detail.htm?menuKey=4&kycOrderId="

# Function to concatenate URL with order ID and make it a full link
def create_clickable_link(order_id):
    full_url = (f"{base_url}{order_id}&failedReason=There%20was%20a%20problem%20while%20upgrading."
                "&submissionDate=1669465310788&modifiedDate=1669465320278"
                "&orderStatus=FAILED&loginId=62-85156229291&userNickname=MUHAMMAD%20ARIES%20RAMADHAN")
    return f'<a href="{full_url}" target="_blank">Link</a>'

# Adding a new column 'Link' that contains clickable URLs
telpon['Link'] = telpon['ORDER ID'].apply(create_clickable_link)

# Select only the 'ORDER ID' and 'Link' columns for display
display_columns = telpon[['ORDER ID', 'Link']]

# Display the DataFrame with clickable links (now with the index)
display(display_columns.style.format({'Link': lambda x: x}).set_properties(**{'text-align': 'left'}))

#========================== to print
# Convert the 'ORDER ID' column to a list
order_id_list = telpon['ORDER ID'].tolist()

# Join the list items into a single string separated by commas
order_ids_string = ", ".join(order_id_list)

print("")
# Print the result
print(order_ids_string)

,ORDER ID,Link
260115,20240609111215800000DANAW3ID035649259761,Link



20240609111215800000DANAW3ID035649259761


In [ ]:

def change_telpon_bulk(df):
    changed_rows = set()

    # Prompt the user for Order IDs
    order_ids_input = input("Enter all Order IDs separated by commas (or type 'No' to stop): ").strip()

    # Check if the user wants to stop
    if order_ids_input.lower() == 'no':
        print("Stopping the process.")
        return

    # Split the input into a list of Order IDs
    order_ids = [order_id.strip() for order_id in order_ids_input.split(',')]

    for order_id_to_change in order_ids:
        # Prompt the user for the new Status value for each Order ID
        new_status_value = input(f"Enter the new No Telepon value for ORDER ID {order_id_to_change}: ").strip()

        # Locate the row with the specified "ORDER ID"
        changed_row = df[df['ORDER ID'] == order_id_to_change].index

        if changed_row.empty:
            print(f"\nNo row found for ORDER ID: {order_id_to_change}")
        else:
            # Update the "STATUS" value
            df.loc[changed_row, 'NO. HP'] = new_status_value
            ori_df.loc[changed_row, 'NO. HP'] = new_status_value
            # Add the changed row to the set
            changed_rows.add(changed_row[0])

            # Display only the 'ORDER ID' and 'STATUS' columns for the changed row
            # print(f"\nChanged Row for ORDER ID {order_id_to_change}:\n{df.loc[changed_row, ['ORDER ID', 'STATUS']]}")

    # Display all changed rows after stopping the process
    if changed_rows:
        print("\nAll Changed Rows:")
        #print(df.loc[changed_rows, ['ORDER ID', 'NO. HP']])

# Example usage:
change_telpon_bulk(df)

Enter all Order IDs separated by commas (or type 'No' to stop): 20240609111215800000DANAW3ID035649259761
Enter the new No Telepon value for ORDER ID 20240609111215800000DANAW3ID035649259761: 6289663116630

All Changed Rows:


## Mengganti Status

In [ ]:
pd.set_option('display.max_rows', None)
status = df[(df['Keterangan'].str.contains('check status', na=False)) &
            (df['Modified Date'].notnull()) &
            (~df['STATUS'].str.contains('REJECT BY SYSTEM', na=False))
            ]
sorted_status = status.sort_values(by='STATUS')
# sorted_status[['ID AGENT', 'ORDER ID', 'NO. HP', 'STATUS', 'Reason', 'Modified Date', 'WorkingDate', 'Keterangan']]


In [ ]:
# Define a base URL
base_url = "https://custmng.saas.dana.id/user/kyc/detail.htm?menuKey=4&kycOrderId="

# Function to concatenate URL with order ID and make it a full link
def create_clickable_link(order_id):
    full_url = (f"{base_url}{order_id}&failedReason=There%20was%20a%20problem%20while%20upgrading."
                "&submissionDate=1669465310788&modifiedDate=1669465320278"
                "&orderStatus=FAILED&loginId=62-85156229291&userNickname=MUHAMMAD%20ARIES%20RAMADHAN")
    return f'<a href="{full_url}" target="_blank">Link</a>'

# Adding a new column 'Link' that contains clickable URLs
sorted_status['Link'] = sorted_status['ORDER ID'].apply(create_clickable_link)

# Select only the 'ORDER ID' and 'Link' columns for display
display_columns = sorted_status[['ORDER ID', 'STATUS', 'Reason','Link','Modified Date', 'WorkingDate', 'Keterangan']]

# Display the DataFrame with clickable links (now with the index)
display(display_columns.style.format({'Link': lambda x: x}).set_properties(**{'text-align': 'left'}))

print("Jumlah Row:  ",len(sorted_status))

,ORDER ID,STATUS,Reason,Link,Modified Date,WorkingDate,Keterangan
239204,20240607111215800000DANAW3ID736947529667,CHECK,SELFIE BERBEDA,Link,2024-06-08 04:51:23,2024-06-08 00:00:00,check status
283565,20240610111215800000DANAW3ID724649577419,CHECK,SELFIE BERBEDA,Link,2024-06-10 15:19:43,2024-06-10 00:00:00,check status
507583,20240610111215800000DANAW3ID978949149843,CHECK,NIK PUDAR,Link,2024-06-10 22:16:09,2024-06-10 00:00:00,check status
287804,20240610111215800000DANAW3ID935149309954,REJECT,nan,Link,2024-06-10 16:35:15,2024-06-10 00:00:00,check status
238981,20240607111215800000DANAW3ID084947852646,REVOKED,nan,Link,2024-06-08 01:41:13,2024-06-08 00:00:00,check status
200325,20240609111215800000DANAW3ID487750241174,nan,nan,Link,2024-06-09 22:36:09,2024-06-09 00:00:00,check status


Jumlah Row:   6


In [ ]:
# Dictionary of changes (ORDER ID as keys, values as dictionaries of new STATUS and KETERANGAN)
changes = {

    '20240607111215800000DANAW3ID736947529667': {'STATUS': 'CHECK', 'Keterangan': 'reason salah'},

    '20240610111215800000DANAW3ID724649577419': {'STATUS': 'REJECT', 'Keterangan': 'finding salah status'},

    '20240610111215800000DANAW3ID978949149843': {'STATUS': 'REVOKED', 'Keterangan': 'finding salah status'},

    '20240610111215800000DANAW3ID935149309954': {'STATUS': 'REJECT', 'Keterangan': 'reason salah'},

    '20240607111215800000DANAW3ID084947852646': {'STATUS': 'REVOKED', 'Keterangan': 'reason salah'},

    '20240609111215800000DANAW3ID487750241174': {'STATUS': 'CHECK', 'Keterangan': 'finding salah status'}

    # '20240601111215800000DANAW3ID516142666773': {'STATUS': 'CHECK', 'Keterangan': 'status salah'}

    # '20240604111215800000DANAW3ID469046215638': {'STATUS': 'REJECT', 'Keterangan': 'reason salah'},

    # '20240605111215800000DANAW3ID844846434973': {'STATUS': 'REJECT', 'Keterangan': 'reason salah'},

    # '20240605111215800000DANAW3ID057346281920': {'STATUS': 'REJECT', 'Keterangan': 'reason salah'},

    # '20240605111215800000DANAW3ID363346052667': {'STATUS': 'REJECT', 'Keterangan': 'reason salah'},

    # '20240606111215800000DANAW3ID333546844649': {'STATUS': 'REJECT', 'Keterangan': 'reason salah'},

    #  '20240604111215800000DANAW3ID363145462611': {'STATUS': 'REJECT', 'Keterangan': 'reason salah'},

    # '20240606111215800000DANAW3ID079646915111': {'STATUS': 'REJECT', 'Keterangan': 'reason salah'},

    # '20240605111215800000DANAW3ID503345352460': {'STATUS': 'REVOKED', 'Keterangan': 'reason salah'},

    # '20240601111215800000DANAW3ID516142666773': {'STATUS': 'CHECK', 'Keterangan': 'status salah'}

    # '20240425111215800000DANAW3ID202716246645': {'STATUS': 'REVOKED', 'Keterangan': 'reason salah'},

    # '20240425111215800000DANAW3ID272816104513': {'STATUS': 'REVOKED', 'Keterangan': 'reason salah'},

    # '20240425111215800000DANAW3ID433616290220': {'STATUS': 'CHECK', 'Keterangan': 'finding salah status'}

    # '20240408111215800000DANAW3ID614603026202': {'STATUS': 'CHECK', 'Keterangan': 'status salah'},

    #  '20240408111215800000DANAW3ID617902699939': {'STATUS': 'CHECK', 'Keterangan': 'status salah'},

    #  '20240410111215800000DANAW3ID531404307400': {'STATUS': 'CHECK', 'Keterangan': 'status salah'},

    #  '20240410111215800000DANAW3ID042805187313': {'STATUS': 'CHECK', 'Keterangan': 'status salah'}

    # '20240228111215800000DANAW3ID377074194144': {'STATUS': 'REVOKED', 'Keterangan': 'reason salah'},

    # '20240228111215800000DANAW3ID033674059763': {'STATUS': 'REVOKED', 'Keterangan': 'reason salah'}

    # '20240224111215800000DANAW3ID577370196920': {'STATUS': 'REVOKED', 'Keterangan': 'reason salah'},


}
# Apply the changes in one line
for order_id, values in changes.items():
    df.loc[df['ORDER ID'] == order_id, ['STATUS', 'Keterangan']] = [values['STATUS'], values['Keterangan']]


Function For Checking

In [ ]:
# pd.set_option('display.max_rows', None)
# status = df[df['ORDER ID'].str.contains('20231201111215800000DANAW3ID970814217454',na=False)]
# status[['ORDER ID','STATUS','Keterangan']]



Berhasil mengganti status secara multiple

## Modified Date & Working Date

In [ ]:
pd.set_option('display.max_rows', None)
modif = df[df['Keterangan'].str.contains('modified',na=False)]
modif.head(5)



,ORDER ID,NO. HP,STATUS,Reason,New Remarks,ID AGENT,NAMA AGENT,Modified Date,Support DOC,Face Score,OCR,Biometric Score,WorkingDate,KTP Defect >1,Status Revoke,Reason Revoke,New Remarks Revoke,Unnamed: 17,Keterangan
677,20240502111215800000DANAW3ID981820510715,6285922341014,REJECT BY SYSTEM,NaN,NaN,agent.dte1200@radikari.com,Adi Alfaridzi,NaT,Tidak Ada,nan,Sesuai,0.0,2024-05-02,NaN,NaN,NaN,NaN,NaN,modified date salah
3815,20240511111215800000DANAW3ID503427183416,62895370997346,REJECT BY SYSTEM,NaN,NaN,agent.dte1200@radikari.com,Adi Alfaridzi,NaT,Tidak Ada,nan,Sesuai,0.0,2024-05-11,NaN,NaN,NaN,NaN,NaN,modified date salah
3898,20240511111215800000DANAW3ID272828466976,6285735567054,REJECT BY SYSTEM,NaN,NaN,agent.dte1200@radikari.com,Adi Alfaridzi,NaT,Tidak Ada,nan,Sesuai,0.0,2024-05-11,NaN,NaN,NaN,NaN,NaN,modified date salah
5147,20240501111215800000DANAW3ID165721464392,62817555900,REJECT BY SYSTEM,NaN,NaN,agent.dte1201@radikari.com,Ananda Aisyah,NaT,Tidak Ada,73.0,Sesuai,0.0,2024-05-01,NaN,NaN,NaN,NaN,NaN,modified date salah
6849,20240505111215800000DANAW3ID381823206863,6285659863644,REJECT BY SYSTEM,NaN,NaN,agent.dte1201@radikari.com,Ananda Aisyah,NaT,Tidak Ada,nan,Sesuai,0.0,2024-05-05,NaN,NaN,NaN,NaN,NaN,modified date salah


#Processing Data 2

In [ ]:
# Replace NaN with empty string and convert to string
df['Reason'] = df['Reason'].fillna('').astype(str)
df['New Remarks'] = df['New Remarks'].fillna('').astype(str)
df['KTP Defect >1'] = df['KTP Defect >1'].fillna('').astype(str)
df['Status Revoke'] = df['Status Revoke'].fillna('').astype(str)
df['Reason Revoke'] = df['Reason Revoke'].fillna('').astype(str)
df['New Remarks Revoke'] = df['New Remarks Revoke'].fillna('').astype(str)

In [ ]:
# Remove Whitespace
df['Status Revoke'] = df['Status Revoke'].str.strip()
df['Reason Revoke'] = df['Reason Revoke'].str.strip()
df['New Remarks Revoke'] = df['New Remarks Revoke'].str.strip()

In [ ]:
df.tail(2)

,ORDER ID,NO. HP,STATUS,Reason,New Remarks,ID AGENT,NAMA AGENT,Modified Date,Support DOC,Face Score,OCR,Biometric Score,WorkingDate,KTP Defect >1,Status Revoke,Reason Revoke,New Remarks Revoke,Keterangan
590234,20240610111215800000DANAW3ID368749792427,6285819801074.0,CHECK,,,cakratriatmaja@radikari.com,Cakra Triatmaja,2024-06-10 23:59:28,KK,64.0,Sesuai,0.0,2024-06-10,,,,,None
590235,20240610111215800000DANAW3ID272151133686,6282111271969.0,CHECK,,,cakratriatmaja@radikari.com,Cakra Triatmaja,2024-06-10 23:59:57,Tidak Ada,72.0,Sesuai,0.0,2024-06-10,,,,,None


##Check Status Revoke, Reject Reason, New Remarks Reason

In [ ]:
# Select rows where STATUS is 'REJECT' or 'REVOKE'
reject_revoke = df[(df['STATUS'] == 'REJECT') | (df['STATUS'] == 'REVOKED')]

In [ ]:
#Ada problem dengan reason revoke inconsisten, mengisi ulang nilai baru dengan kategori ini
pd.set_option('display.max_rows', None)
reject_revoke.loc[reject_revoke['Reason Revoke'].str.contains('INCONSISTENT', case=False, na=False), 'Reason Revoke'] = 'INCONSISTENT KTP (KTP TIDAK KONSISTEN)'
# status[['ORDER ID','STATUS','Status Revoke','Reason Revoke','New Remarks Revoke','Keterangan']]

In [ ]:
#Ada problem dengan reason revoke inconsisten, mengisi ulang nilai baru dengan kategori ini
pd.set_option('display.max_rows', None)
df.loc[df['Reason Revoke'].str.contains('INCONSISTENT', case=False, na=False), 'Reason Revoke'] = 'INCONSISTENT KTP (KTP TIDAK KONSISTEN)'
# status[['ORDER ID','STATUS','Status Revoke','Reason Revoke','New Remarks Revoke','Keterangan']]

In [ ]:
#reject reason list
valid_reasons_revoke = [
    'YOUR ID CARD IS NOT EKTP/KTP',
    'YOUR ID CARD PICTURE IS NOT CLEAR',
    'THE ID NUMBER THAT YOU TYPED DOES NOT MATCH WITH THE ID NUMBER IN THE PICTURE',
    'YOUR ID CARD IS FAKE OR YOUR PHOTO IN THE ID IS FAKE',
    "YOUR SELFIE DOESN'T MATCH WITH ID PHOTO",  # Use double quotes around this string
    'YOUR ID CARD IS ALREADY EXPIRED',
    'YOUR KTP BACKGROUND COLOR DOES NOT MATCH',
    'YOU REQUEST TO REVOKE KYC',
    'KTP ON SCREEN (KTP ADA DI LAYAR)',
    'KTP DAMAGE (RUSAK)',
    'KTP PRINT OUT (KTP CETAKAN)',
    'INCONSISTENT KTP (KTP TIDAK KONSISTEN)',
    'TERDAPAT LEBIH DARI SATU ORANG PADA FOTO SELFIE',
    'FETCHING'
]

valid_reasons_reject = [
  "Reject Document Is not eKTP / KTP (Dokumen bukan eKTP / KTP)",
  "Reject Document is Overexposed (Foto Dokumen Terlalu Terang)",
  "Reject Document is Underexposed (Foto Dokumen Terlalu Gelap)",
  "Reject Document is Not Clear and Blured (Foto Dokumen Tidak Jelas & Tidak Bisa Dibaca)",
  "Reject ID Card is Broken (Dokumen Rusak)",
  "Reject ID Number is not same with User Input DATA CLEAR (Nomor KTP Tidak Sesuai yang Di-Input User data jelas)",
  "Reject ID Number is not same with User Input DATA UNCLEAR (Nomor KTP Tidak Sesuai yang Di-Input User data tidak jelas)",
  "Reject Picture in the ID is a Fake (KTP Editan)",
  "Reject Selfie doesn't match with ID Photo (Selfie tidak Sesuai dengan Foto KTP)",
  "Reject ID is Expired (Dokumen Expired)",
  "Reject Supdoc type is not acceptable (SIM, Passport, KK, Ijazah, Buku Nikah). (Jenis dokumen pendukung tidak sesuai ketentuan (SIM, Paspor, KK, Buku Nikah, Ijazah).)"

  # #Bulan Juni Hapus dibawah ini
  # "Reject Document is Overexposure (Foto Dokumen Terlalu Terang)",
  # "Reject Document is Underexposure (Foto Dokumen Terlalu Gelap)",
  # "Reject Document is Clear and Not Blured (Foto Dokumen Jelas & Bisa Dibaca)",
  # "Reject Document is Not Clear and Blured (Foto Dokumen Jelas & Bisa Dibaca)",
  # "Reject ID Card not Intact (Dokumen Tidak Rusak)",
  # "Reject Picture in the ID is a Fake (Foto pada KTP Tempelan / Editan)",
  # "Reject Selfie doesn't matches with ID Photo (Selfie tidak Sesuai dengan Foto KTP)"
]


In [ ]:
#new remarks revoke
remark_list = [
    'Abnormal Background (Background foto KTP tidak seperti biasa)',
    'Background KTP doesn’t match (Background Foto KTP Tidak Sesuai)',
    'Biometric Score <6',
    'Biometric Score <8',
    'DOB Not Match With DOP Id Card (TTL Tidak Sesuai Dengan Tahun Pembuatan)',
    'Document is Overexposure (Foto Dokumen Terlalu Terang)',
    'Edited Id Card (KTP Editan)',
    'Id Card Damage (KTP Rusak)',
    'Id Card Image Blur (KTP Blur)',
    'Inconsistent Data, > 4 Id Card (Data Inkonsisten, History lebih dari 4)',
    'On Screen Id Card (KTP On Screen)',
    'Photo On Id Card Taped (Foto KTP Tempelan)',
    'Photocopy ID Card (KTP Fotokopi)',
    'Print Id Card (KTP Print)',
    'Selfie Doesn’t Match With Id Card (Selfie tidak sesuai)',
    'Support Document Inappropriate (Dokumen pendukung tidak sesuai)', #delet ini 2024
    'Support Document Vulgar (Dokumen Pendukung Vulgar)',
    'Support Document Invalid (Dokumen Pendukung Tidak Sesuai)',
    'Suspicious ID Network High Risk',
    'Suspicious Ilegal Letters High Risk',
    'Suspicious Screen Retaken High Risk',
    'Suspicious Velocity High Risk',
    'The Id Number That You Typed Does Not Match With The Id Number In The Picture (NIK Typo)',
    'Your Id Card Is Already Expired (KTP Expired)',
    'Your Id Card Is Not Ektp/Ktp (Bukan E-KTP/KTP)',
    'Your Id Card Picture Is Not Clear (Tertutup, Terkena Noda & Ditebalkan)'
]

In [ ]:
def status_revoke(row):
    if row['STATUS'] in ['REJECT', 'REVOKED'] and pd.isna(row['Status Revoke']) or row['Status Revoke'] not in ['REVOKED', 'REVOKED & Apply KYC']:
      # Update 'Status Revoke' to 'REVOKED'
      df.loc[row.name, 'Status Revoke'] = 'REVOKED'
      if pd.isna(row['Keterangan']):
        return 'status revoke salah'
      else:
        return f"{row['Keterangan']} & status revoke salah"
    else:
      return row['Keterangan']

In [ ]:
def reason_revoke (row):
    if row['STATUS'].lower() in ['reject', 'revoked'] and pd.isna(row['Reason Revoke']):
    #if row['STATUS'] in ['REJECT', 'REVOKED'] and pd.isna(row['Reason Revoke']) or row['Reason Revoke'] not in valid_reasons:
      if pd.isna(row['Keterangan']):
        return 'reason revoke salah'
      else:
        return f"{row['Keterangan']} & reason revoke salah"

    elif row['STATUS'].lower() in ['reject'] and row['Reason Revoke'].lower() not in map(str.lower, valid_reasons_reject):
      if pd.isna(row['Keterangan']):
        return 'reason revoke salah'
      else:
        return f"{row['Keterangan']} & reason revoke salah"

    elif row['STATUS'].lower() in ['revoke'] and row['Reason Revoke'].lower() not in map(str.lower, valid_reasons_revoke):
      if pd.isna(row['Keterangan']):
        return 'reason revoke salah'
      else:
        return f"{row['Keterangan']} & reason revoke salah"

    else:
        return row['Keterangan']

In [ ]:
def nr_revoke (row):
    if row['STATUS'].lower() in ['reject', 'revoked'] and pd.isna(row['New Remarks Revoke']) or row['New Remarks Revoke'].lower() not in map(str.lower, remark_list):
    #if row['STATUS'] in ['REJECT', 'REVOKED'] and pd.isna(row['New Remarks Revoke']) or row['New Remarks Revoke'] not in remark_list:
      if pd.isna(row['Keterangan']):
        return 'new remarks revoke salah'
      else:
        return f"{row['Keterangan']} & new remarks revoke salah"

    else:
        return row['Keterangan']

In [ ]:
# Apply the custom function to update "Keterangan" column
reject_revoke['Keterangan'] = reject_revoke.apply(status_revoke, axis=1)
reject_revoke['Keterangan'] = reject_revoke.apply(reason_revoke, axis=1)
reject_revoke['Keterangan'] = reject_revoke.apply(nr_revoke, axis=1)

In [ ]:
x = reject_revoke[reject_revoke['Keterangan'].str.contains('Revoke', case=False, na=False)]
x[['ORDER ID','NO. HP','STATUS','Status Revoke','Reason Revoke','New Remarks Revoke','Keterangan']]

,ORDER ID,NO. HP,STATUS,Status Revoke,Reason Revoke,New Remarks Revoke,Keterangan
45733,20240610111215800000DANAW3ID335249291460,6285795263909,REVOKED,,,,status revoke salah & new remarks revoke salah
45796,20240610111215800000DANAW3ID897549724126,6282368216470,REJECT,REVOKED,YOUR ID CARD PICTURE IS NOT CLEAR,Id Card Image Blur (KTP Blur),reason revoke salah
142811,20240610111215800000DANAW3ID781648980451,6285793830357,REJECT,FAILED REVOKED,Reject Document Is not eKTP / KTP (Dokumen buk...,Print Id Card (KTP Print),status revoke salah
173298,20240609111215800000DANAW3ID624449520646,6282210611437,REJECT,REVOKED,KTP DAMAGE (RUSAK),Id Card Damage (KTP Rusak),reason revoke salah
204069,20240610111215800000DANAW3ID044350359816,6287812294974,REJECT,REVOKED,YOUR SELFIE DOESN'T MATCH WITH ID PHOTO,Selfie Doesn’t Match With Id Card (Selfie tida...,reason revoke salah
207630,20240609111215800000DANAW3ID251250246063,6282196771988,REJECT,REVOKED,KTP DAMAGE (RUSAK),Id Card Damage (KTP Rusak),reason revoke salah
238981,20240607111215800000DANAW3ID084947852646,6285229097824,REVOKED,,,,reason salah & status revoke salah & new remar...
239662,20240608111215800000DANAW3ID358148640075,628979653929,REJECT,REVOKED,Reject Selfie doesn't match with ID Photo (Sel...,,new remarks revoke salah
264265,20240609111215800000DANAW3ID927448518487,6285270483013,REJECT,,,,status revoke salah & reason revoke salah & ne...
283665,20240610111215800000DANAW3ID546949280551,6283132965973,REJECT,REVOKED,Reject Selfie doesn't match with ID Photo (Sel...,selfi,new remarks revoke salah


#Finishing Data

In [ ]:
# Remove values in 'Keterangan' column where 'STATUS' is "REJECT BY SYSTEM"
df.loc[df['STATUS'] == 'REJECT BY SYSTEM', 'Keterangan'] = np.nan
df.loc[df['Reason'] == 'REJECT BY SYSTEM', 'Keterangan'] = np.nan

In [ ]:
# Combine the values from both dataframes into a single column
df['Keterangan'] = df['Keterangan'].combine_first(reject_revoke['Keterangan'])
ori_df['Keterangan'] = ori_df['Keterangan'].combine_first(df['Keterangan'])
ori_df['STATUS'] = ori_df['STATUS'].combine_first(df['STATUS'])
ori_df['ID AGENT'] = ori_df['ID AGENT'].combine_first(df['ID AGENT'])
ori_df['NAMA AGENT'] = ori_df['NAMA AGENT'].combine_first(df['NAMA AGENT'])


In [ ]:
# # Perform left join to bring updated "NO. HP" values
# temp_df = ori_df.merge(df[['ORDER ID', 'NO. HP']], on='ORDER ID', how='left', suffixes=('', '_updated'))

# # Update "NO. HP" in ori_df with the values from df
# ori_df['NO. HP'] = temp_df['NO. HP_updated'].combine_first(temp_df['NO. HP'])

In [ ]:
ori_df.tail(100)

,ORDER ID,NO. HP,STATUS,Reason,New Remarks,ID AGENT,NAMA AGENT,Modified Date,Support DOC,Face Score,OCR,Biometric Score,WorkingDate,KTP Defect >1,Status Revoke,Reason Revoke,New Remarks Revoke,Keterangan
590440,20240610111215800000DANAW3ID847150796380,6282332680206.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,2024-06-11 03:39:43,Tidak Ada,70.0,Tidak Sesuai,0.0,2024-06-11,NaN,NaN,NaN,NaN,NaN
590441,20240610111215800000DANAW3ID858550571743,6282279016642.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,2024-06-11 03:40:18,Tidak Ada,73.0,Sesuai,0.0,2024-06-11,NaN,NaN,NaN,NaN,NaN
590442,20240610111215800000DANAW3ID250350634320,6281345157236.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,2024-06-11 03:41:21,Tidak Ada,73.0,Tidak Sesuai,0.0,2024-06-11,NaN,NaN,NaN,NaN,NaN
590443,20240610111215800000DANAW3ID777249639818,6285765670100.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,2024-06-11 03:41:59,Tidak Ada,71.0,Sesuai,0.0,2024-06-11,NaN,NaN,NaN,NaN,NaN
590444,20240610111215800000DANAW3ID683749873944,6289531069168.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,2024-06-11 03:42:31,Tidak Ada,69.0,Sesuai,0.0,2024-06-11,NaN,NaN,NaN,NaN,NaN
590445,20240610111215800000DANAW3ID372349889054,6281362926966.0,REJECT,SELFIE BERBEDA,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,2024-06-11 03:43:38,KK,58.0,Sesuai,0.0,2024-06-11,No,REVOKED,Reject Selfie doesn't match with ID Photo (Sel...,Selfie Doesn’t Match With Id Card (Selfie tida...,NaN
590446,20240610111215800000DANAW3ID266050619150,6285260659579.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,2024-06-11 03:44:19,Tidak Ada,71.0,Sesuai,0.0,2024-06-11,NaN,NaN,NaN,NaN,NaN
590447,20240610111215800000DANAW3ID862150402280,62895329346998.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,2024-06-11 03:44:49,Tidak Ada,72.0,Sesuai,0.0,2024-06-11,NaN,NaN,NaN,NaN,NaN
590448,20240610111215800000DANAW3ID228651071073,6287877441618.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,2024-06-11 03:45:30,Tidak Ada,77.0,Tidak Sesuai,0.0,2024-06-11,NaN,NaN,NaN,NaN,NaN
590449,20240610111215800000DANAW3ID441150746506,6281218172920.0,CHECK,NaN,NaN,cakratriatmaja@radikari.com,Cakra Triatmaja,2024-06-11 03:46:24,Tidak Ada,68.0,Tidak Sesuai,0.0,2024-06-11,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Get a list of columns containing 'Unnamed'
unnamed_columns = [col for col in df.columns if 'Unnamed' in col]

# Drop the columns with 'Unnamed' in their names
df = df.drop(columns=unnamed_columns)
ori_df = ori_df.drop(columns=unnamed_columns)


##Monitoring Pengerjaan Agent

In [ ]:
# Filter rows based on conditions and set 'Keterangan' to null
df.loc[(df['STATUS'].isnull()) & (df['Modified Date'].isnull()) & (df['WorkingDate'] == end_date), 'Keterangan'] = None

In [ ]:
# Set display option to show all rows
pd.set_option('display.max_rows', None)

Data di bawah ini masih ada dupliacte by order id

In [ ]:
filtered_df = df[
    # (df['STATUS'].notnull()) &
    # (df['Reason'].notnull()) &
    # (df['Modified Date'].notnull()) &
    (df['Keterangan'].notnull() & ~df['Keterangan'].str.contains('finding', case=False, na=False))
]

# Format the 'Modified Date' column to 'dd-mm-yyyy hh:mm:ss'
filtered_df['Modified Date'] = filtered_df['Modified Date'].dt.strftime('%d-%m-%Y %H:%M:%S')
filtered_df['WorkingDate'] = filtered_df['WorkingDate'].dt.strftime('%d-%m-%Y')

# First, convert the 'Face Score' column to numeric (float)
filtered_df['Face Score'] = pd.to_numeric(filtered_df['Face Score'], errors='coerce')

# Now, convert to integer
filtered_df['Face Score'] = filtered_df['Face Score'].astype('Int64')
filtered_df['Face Score'] = filtered_df['Face Score'].astype('str')

# Replace NaN or NaT with blank (empty string)
filtered_df.replace({np.nan: '', pd.NaT: ''}, inplace=True)

# Drop duplicates in the 'ORDER ID' column
filtered_df_no_duplicates = filtered_df.drop_duplicates(subset='ORDER ID')

# Remove values in 'Keterangan' column where 'STATUS' is "REJECT BY SYSTEM"
filtered_df_no_duplicates.loc[filtered_df_no_duplicates['STATUS'] == 'REJECT BY SYSTEM', 'Keterangan'] = ''
filtered_df_no_duplicates.loc[filtered_df_no_duplicates['Reason'] == 'REJECT BY SYSTEM', 'Keterangan'] = ''

filtered_df_no_duplicates.head(10)



,ORDER ID,NO. HP,STATUS,Reason,New Remarks,ID AGENT,NAMA AGENT,Modified Date,Support DOC,Face Score,OCR,Biometric Score,WorkingDate,KTP Defect >1,Status Revoke,Reason Revoke,New Remarks Revoke,Keterangan
45733,20240610111215800000DANAW3ID335249291460,6285795263909,REVOKED,KTP BLUR,,reckybranada@radikari.com,Recky Branada,10-06-2024 11:07:19,Tidak Ada,72,Sesuai,0.0,10-06-2024,,REVOKED,,,status revoke salah & new remarks revoke salah
45796,20240610111215800000DANAW3ID897549724126,6282368216470,REJECT,KTP BLUR,,reckybranada@radikari.com,Recky Branada,10-06-2024 11:56:50,SIM,65,Sesuai,0.0,10-06-2024,No,REVOKED,YOUR ID CARD PICTURE IS NOT CLEAR,Id Card Image Blur (KTP Blur),reason revoke salah
46118,20240610111215800000DANAW3ID528449678549,6285732273117,REVOKED,KTP TERKENA FLASH,,reckybranada@radikari.com,Recky Branada,,Tidak Ada,75,Sesuai,0.0,10-06-2024,No,REVOKED,YOUR ID CARD PICTURE IS NOT CLEAR,Document is Overexposure (Foto Dokumen Terlalu...,modified date salah
142811,20240610111215800000DANAW3ID781648980451,6285793830357,REJECT,KTP PRINT,,amandarapih@radikari.com,Amanda Rapih Subarja Putri,10-06-2024 12:28:07,KK,65,Sesuai,0,10-06-2024,No,REVOKED,Reject Document Is not eKTP / KTP (Dokumen buk...,Print Id Card (KTP Print),status revoke salah
173298,20240609111215800000DANAW3ID624449520646,6282210611437,REJECT,NAMA PUDAR,,agent.jogja912@radikari.com,Dea Fatika Putri,10-06-2024 08:27:28,KK,66,Tidak Sesuai,0.0,10-06-2024,No,REVOKED,KTP DAMAGE (RUSAK),Id Card Damage (KTP Rusak),reason revoke salah
204069,20240610111215800000DANAW3ID044350359816,6287812294974,REJECT,SELFIE BERBEDA,,agent.jogja932@radikari.com,Irvan Susilo Widjiatmoko,10-06-2024 21:27:48,SIM,59,Tidak Sesuai,,10-06-2024,No,REVOKED,YOUR SELFIE DOESN'T MATCH WITH ID PHOTO,Selfie Doesn’t Match With Id Card (Selfie tida...,reason revoke salah
207630,20240609111215800000DANAW3ID251250246063,6282196771988,REJECT,KTP RUSAK,,agent.jogja937@radikari.com,Andreas Suryo Prasetyo,10-06-2024 18:37:19,KK,63,Sesuai,0,10-06-2024,No,REVOKED,KTP DAMAGE (RUSAK),Id Card Damage (KTP Rusak),reason revoke salah
238981,20240607111215800000DANAW3ID084947852646,6285229097824,REVOKED,,,agent.jogja955@radikari.com,Nova Putra Alves Correia,08-06-2024 01:41:13,Tidak Ada,71,Tidak Sesuai,0.0,08-06-2024,,REVOKED,,,reason salah
239204,20240607111215800000DANAW3ID736947529667,6285832023157,CHECK,SELFIE BERBEDA,,agent.jogja955@radikari.com,Nova Putra Alves Correia,08-06-2024 04:51:23,KK,57,Tidak Sesuai,0.0,08-06-2024,,,,,reason salah
239662,20240608111215800000DANAW3ID358148640075,628979653929,REJECT,SELFIE BERBEDA,,agent.jogja955@radikari.com,Nova Putra Alves Correia,09-06-2024 02:52:33,KK,57,Tidak Sesuai,0.0,09-06-2024,No,REVOKED,Reject Selfie doesn't match with ID Photo (Sel...,,new remarks revoke salah


In [ ]:
filtered_df_no_duplicates.to_excel('monitoring.xlsx', index=False)
# # # Download the file
files.download('monitoring.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Finding

In [ ]:
# Apply filters
finding = df[df['Keterangan'].str.contains('finding', case=False, na=False)]

# First, convert the 'Face Score' column to numeric (float)
finding['Face Score'] = pd.to_numeric(finding['Face Score'], errors='coerce')

# Now, convert to integer
finding['Face Score'] = finding['Face Score'].astype('Int64')
finding['Face Score'] = finding['Face Score'].astype('str')

finding.replace({np.nan: '', pd.NaT: ''}, inplace=True)
finding

,ORDER ID,NO. HP,STATUS,Reason,New Remarks,ID AGENT,NAMA AGENT,Modified Date,Support DOC,Face Score,OCR,Biometric Score,WorkingDate,KTP Defect >1,Status Revoke,Reason Revoke,New Remarks Revoke,Keterangan
200325,20240609111215800000DANAW3ID487750241174,628895804456,CHECK,,,agent.jogja928@radikari.com,Donie Pramudita,2024-06-09 22:36:09,Tidak Ada,73,Sesuai,0,2024-06-09,,,,,finding salah status
283565,20240610111215800000DANAW3ID724649577419,6287728421285,REJECT,SELFIE BERBEDA,,ahmadkarim@radikari.com,Ahmad Busyrol Karim,2024-06-10 15:19:43,Tidak Ada,57,Tidak Sesuai,0.0,2024-06-10,No,REVOKED,Reject Selfie doesn't match with ID Photo (Sel...,Selfie Doesn’t Match With Id Card (Selfie tida...,finding salah status
507583,20240610111215800000DANAW3ID978949149843,6285823051837,REVOKED,NIK PUDAR,,agent.srg523@radikari.com,Argia Asmara Dewanggoro,2024-06-10 22:16:09,Tidak Ada,71,Sesuai,0.0,2024-06-10,,REVOKED,,,finding salah status


In [ ]:
finding.to_excel('finding.xlsx', index=False)
# # Download the file
files.download('finding.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Backup Revoke

In [ ]:
# Get user input for WorkingDate
user_input = input("Enter WorkingDate (yyyy-mm-dd) : ")

# Filter DataFrame based on conditions
Revoke = df[(df['STATUS'].isin(['REVOKED', 'REJECT'])) & (df['WorkingDate'] == user_input)]

# Sort the filtered DataFrame by 'Reason Revoke' ascending
sorted_df = Revoke.sort_values(by='Reason', ascending=True)

# Format the 'Modified Date' column to 'dd-mm-yyyy hh:mm:ss'
sorted_df['Modified Date'] = sorted_df['Modified Date'].dt.strftime('%d-%m-%Y %H:%M:%S')
sorted_df['WorkingDate'] = sorted_df['WorkingDate'].dt.strftime('%d-%m-%Y')

# First, convert the 'Face Score' column to numeric (float)
sorted_df['Face Score'] = pd.to_numeric(sorted_df['Face Score'], errors='coerce')

# Now, convert to integer
sorted_df['Face Score'] = sorted_df['Face Score'].astype('Int64')
sorted_df['Face Score'] = sorted_df['Face Score'].astype('str')

# Replace NaN or NaT with blank (empty string)
sorted_df.replace({np.nan: '', pd.NaT: ''}, inplace=True)

# Sort the filtered DataFrame by 'Reason Revoke' ascending
sorted_df = Revoke.sort_values(by='Reason', ascending=True)

sorted_df.head()

Enter WorkingDate (yyyy-mm-dd) : 2024-6-10


,ORDER ID,NO. HP,STATUS,Reason,New Remarks,ID AGENT,NAMA AGENT,Modified Date,Support DOC,Face Score,OCR,Biometric Score,WorkingDate,KTP Defect >1,Status Revoke,Reason Revoke,New Remarks Revoke,Keterangan
287804,20240610111215800000DANAW3ID935149309954,6285850462938,REJECT,,,miftahulqoyim@radikari.com,Muhammad Miftahul Qoyim,2024-06-10 16:35:15,KK,56.0,Sesuai,0.0,2024-06-10,No,REVOKED,Reject Selfie doesn't match with ID Photo (Sel...,Selfie Doesn’t Match With Id Card (Selfie tida...,reason salah
450205,20240610111215800000DANAW3ID394549574011,6282159958945,REVOKED,AGAMA DICORET,,mayhusnaino@radikari.com,Mayhusna Ino Tantyasari,2024-06-10 12:03:10,Tidak Ada,75,Sesuai,0.0,2024-06-10,No,REVOKED,KTP DAMAGE (RUSAK),"Your Id Card Picture Is Not Clear (Tertutup, T...",None
185379,20240609111215800000DANAW3ID390149169602,6282159958945,REVOKED,AGAMA DITEBALKAN,,agent.jogja918@radikari.com,Randy Aldany Barasa,2024-06-10 05:39:32,Tidak Ada,73.0,Sesuai,0.0,2024-06-10,No,REVOKED,KTP DAMAGE (RUSAK),"Your Id Card Picture Is Not Clear (Tertutup, T...",None
4171,20240610111215800000DANAW3ID768250123676,6281371900464,REVOKED,AGAMA PUDAR,,hanifalvian@radikari.com,Hanif Alvian Rosyiadi,2024-06-10 21:13:57,Tidak Ada,70.0,Sesuai,0,2024-06-10,No,REVOKED,KTP DAMAGE (RUSAK),Id Card Damage (KTP Rusak),None
122177,20240610111215800000DANAW3ID495251189660,6283115252167,REVOKED,AGAMA PUDAR,,khusnulridho@radikari.com,Khusnul Ridho Moh Arif,2024-06-10 15:19:07,Tidak Ada,78.0,Sesuai,0,2024-06-10,No,REVOKED,KTP DAMAGE (RUSAK),Id Card Damage (KTP Rusak),None


In [ ]:
# from google.colab import files
# # Save DataFrame to an Excel file
sorted_df.to_excel('Revoke 10 June.xlsx', index=False)
# # Download the file
files.download('Revoke 10 June.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Data Process

In [ ]:
# Filter out rows where 'ID AGENT' or 'NAMA AGENT' is null
filtered_df = ori_df.dropna(subset=['ID AGENT', 'NAMA AGENT','STATUS'])

# Count occurrences of each ORDER ID grouped by Working Date
order_id_counts = filtered_df.groupby(['WorkingDate'])['ORDER ID'].nunique().reset_index(name='Count')
order_id_counts

,WorkingDate,Count
0,2024-06-01,45409
1,2024-06-02,49748
2,2024-06-03,48281
3,2024-06-04,47770
4,2024-06-05,45895
5,2024-06-06,52554
6,2024-06-07,66337
7,2024-06-08,66194
8,2024-06-09,64179
9,2024-06-10,63485


##All Data

#Save Data

In [ ]:
# Format the 'Modified Date' column to 'dd-mm-yyyy hh:mm:ss'
# ori_df['Modified Date'] = ori_df['Modified Date'].dt.strftime('%d-%m-%Y %H:%M:%S')
# ori_df['WorkingDate'] = ori_df['WorkingDate'].dt.strftime('%d-%m-%Y')

# Format the 'Modified Date' column to 'dd-mm-yyyy hh:mm:ss'
ori_df['Modified Date'] = ori_df['Modified Date'].dt.strftime('%d-%m-%Y %H:%M:%S')
ori_df['WorkingDate'] = ori_df['WorkingDate'].dt.strftime('%d-%m-%Y')

# First, convert the 'Face Score' column to numeric (float)
ori_df['Face Score'] = pd.to_numeric(ori_df['Face Score'], errors='coerce')
# Now, convert to integer
ori_df['Face Score'] = ori_df['Face Score'].astype('Int64')
ori_df['Face Score'] = ori_df['Face Score'].astype('str')

# Replace NaN or NaT with blank (empty string)
ori_df.replace({np.nan: '', pd.NaT: ''}, inplace=True)

# Remove duplicates based on 'ORDER ID' and keep the first occurrence
ori_df = ori_df.drop_duplicates(subset='ORDER ID', keep='first')

ori_df.head(5)

,ORDER ID,NO. HP,STATUS,Reason,New Remarks,ID AGENT,NAMA AGENT,Modified Date,Support DOC,Face Score,OCR,Biometric Score,WorkingDate,KTP Defect >1,Status Revoke,Reason Revoke,New Remarks Revoke,Keterangan
0,20240601111215800000DANAW3ID017442453747,83833991865,CHECK,,,agent.jogja769@radikari.com,Galih Reka Mahendra,01-06-2024 11:03:04,KK,67,Tidak Sesuai,0,01-06-2024,,,,,
1,20240601111215800000DANAW3ID526442394329,85343578803,CHECK,,,agent.jogja511@radikari.com,Hesti Vera,02-06-2024 08:08:09,KK,73,Tidak Sesuai,0,02-06-2024,,,,,
2,20240603111215800000DANAW3ID345644368110,85198562003,CHECK,,,agent.jogja511@radikari.com,Hesti Vera,03-06-2024 11:43:30,KK,75,Sesuai,0,03-06-2024,,,,,
3,20240603111215800000DANAW3ID756844255430,81226168970,CHECK,,,agent.jogja609@radikari.com,Amalia Nurhayati,03-06-2024 12:45:51,KK,77,Tidak Sesuai,0,03-06-2024,,,,,
4,20240604111215800000DANAW3ID270346001519,87812994719,CHECK,,,agent.jogja769@radikari.com,Galih Reka Mahendra,04-06-2024 11:40:19,SIM,63,Sesuai,0,04-06-2024,,,,,


Save 1 File

In [ ]:
# from google.colab import files
# # Save DataFrame to an Excel file
ori_df.to_excel('Check1.xlsx',na_rep='', index=False)

# # Download the file
files.download('Check1.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Save 2 File

In [ ]:
excel_path = '/content/'

# Specify the chunk size (number of rows per Excel file)
chunk_size = len(ori_df) // 2  # Split into two equal parts

# Function to save chunks of DataFrame to Excel
def save_to_excel(dataframe, suffix):
    excel_file = f'{excel_path}Check{suffix}.xlsx'
    dataframe.to_excel(excel_file, index=False)
    print(f"Excel file {excel_file} saved successfully.")

# Split ori_df into two chunks and save to separate Excel files
Check_a = ori_df.iloc[:chunk_size]
Check_b = ori_df.iloc[chunk_size:]

save_to_excel(Check_a, '1')
save_to_excel(Check_b, '2')




Excel file /content/Check1.xlsx saved successfully.
Excel file /content/Check2.xlsx saved successfully.


In [ ]:
# # Download the file
files.download('Check1.xlsx')
files.download('Check2.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Save 3 File

In [ ]:
excel_path = '/content/'

# Specify the chunk size (number of rows per Excel file)
chunk_size = len(ori_df) // 3  # Split into three equal parts

# Function to save chunks of DataFrame to Excel
def save_to_excel(dataframe, suffix):
    excel_file = f'{excel_path}Check{suffix}.xlsx'
    dataframe.to_excel(excel_file, index=False)
    print(f"Excel file {excel_file} saved successfully.")

# Split ori_df into three chunks and save to separate Excel files
Check_a = ori_df.iloc[:chunk_size]
Check_b = ori_df.iloc[chunk_size:2*chunk_size]
Check_c = ori_df.iloc[2*chunk_size:]

save_to_excel(Check_a, '1')
save_to_excel(Check_b, '2')
save_to_excel(Check_c, '3')


Excel file /content/Check1.xlsx saved successfully.
Excel file /content/Check2.xlsx saved successfully.
Excel file /content/Check3.xlsx saved successfully.


In [ ]:
# # Download the file
files.download('Check1.xlsx')
files.download('Check2.xlsx')
files.download('Check3.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>